# `ILookup<TKey,TElement>`

An implementation of `ILookup<TKey,TElement>` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.ilookup-2)] is commonly available via the `Enumerable.ToLookup` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.enumerable.tolookup?view=net-8.0)]:

In [1]:
ILookup<string, string> lookup = new []
{
    new KeyValuePair<string, string>("one", "uno"),
    new KeyValuePair<string, string>("two", "dos"),
    new KeyValuePair<string, string>("one", "uno"),
}
.ToLookup(kv => kv.Key, kv => kv.Value);

lookup

[ [ uno, uno ], [ dos ] ] Count 2 (values) index value 0 [ uno, uno ] Key one (values) [ uno, uno ] 1 [ dos ] Key two (values) [ dos ]

The output above helps us to think of `ILookup<TKey,TElement>` as a type similar to `IGrouping<TKey,TElement>`:

In [2]:
IEnumerable<IGrouping<string, KeyValuePair<string, string>>> groups = new []
{
    new KeyValuePair<string, string>("one", "uno"),
    new KeyValuePair<string, string>("two", "dos"),
    new KeyValuePair<string, string>("one", "uno"),
}
.GroupBy(kv => kv.Key);

groups

[ [ [one, uno], [one, uno] ], [ [two, dos] ] ] (values) index value 0 [ [one, uno], [one, uno] ] Key one (values) index value 0 [one, uno] Key one Value uno 1 [one, uno] Key one Value uno 1 [ [two, dos] ] Key two (values) index value 0 [two, dos] Key two Value dos

But we can see from all the output so far that `ILookup<string, string>` is very different from `IEnumerable<IGrouping<string, KeyValuePair<string, string>>>`. Simultaneously, when we `.Select` the keys of these two types, similarity returns:

In [3]:
groups.Select(g => g.Key)

[ one, two ]

In [4]:
lookup.Select(g => g.Key)

[ one, two ]

Both of the types of `g` have a `.Key` property because both of these types are `IGrouping<TKey,TElement>`. The difference here lies in the type of `TElement`: the `.GroupBy` method emits `TElement` as `KeyValuePair<string, string>` while `.ToLookup` returns `string`.

It might be helpful to think of the output of `.ToLookup` as a ‘flattened’ form of the output of `.GroupBy`, removing redundant key information, streamlining access to values.

## `.GroupBy` does not return a type with an indexer while `.ToLookup` does

The unpleasant output below shows the lack of indexer support for the output of `.GroupBy`:

In [5]:
groups["one"]

Error: (1,1): error CS0021: Cannot apply indexing with [] to an expression of type 'IEnumerable<IGrouping<string, KeyValuePair<string, string>>>'

We see that the output of `.ToLookup` does:

In [6]:
IEnumerable<string> oneValue = lookup["one"];

oneValue

Key,one
(values),"[ uno, uno ]"


The `ILookup<TKey,TElement>.Contains` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.ilookup-2.contains?view=net-8.0#system-linq-ilookup-2-contains(-0))] searches the keys:

In [7]:
lookup.Contains("one")

True

This presence of indexer support (and key search) should inspire us to think of `ILookup<string, string>` as a dictionary. This dictionary will not throw an exception for our data shown above like the `Enumerable.ToDictionary` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.enumerable.todictionary?view=net-8.0)]:

In [8]:
IDictionary<string, string> dictionary = new []
{
    new KeyValuePair<string, string>("one", "uno"),
    new KeyValuePair<string, string>("two", "dos"),
    new KeyValuePair<string, string>("one", "uno"),
}
.ToDictionary(kv => kv.Key, kv => kv.Value);

Error: System.ArgumentException: An item with the same key has already been added. Key: one
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at System.Linq.Enumerable.ToDictionary[TSource,TKey,TElement](TSource[] source, Func`2 keySelector, Func`2 elementSelector, IEqualityComparer`1 comparer)
   at System.Linq.Enumerable.ToDictionary[TSource,TKey,TElement](IEnumerable`1 source, Func`2 keySelector, Func`2 elementSelector, IEqualityComparer`1 comparer)
   at System.Linq.Enumerable.ToDictionary[TSource,TKey,TElement](IEnumerable`1 source, Func`2 keySelector, Func`2 elementSelector)
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderO

We can begin spelunking into complexity by trying to use the `.Distinct` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.enumerable.distinct?view=net-8.0)] to prevent the exception:

In [9]:
IDictionary<string, string> dictionary = new []
{
    new KeyValuePair<string, string>("one", "uno"),
    new KeyValuePair<string, string>("two", "dos"),
    new KeyValuePair<string, string>("one", "uno"),
}
.Distinct()
.ToDictionary(kv => kv.Key, kv => kv.Value);

dictionary

key,value
one,uno
two,dos


But the statement above is vulnerable to data like the following:

In [10]:
new []
{
    new KeyValuePair<string, string>("one", "uno"),
    new KeyValuePair<string, string>("two", "dos"),
    new KeyValuePair<string, string>("one", "1"),
}
.Distinct()
.ToDictionary(kv => kv.Key, kv => kv.Value)

Error: System.ArgumentException: An item with the same key has already been added. Key: one
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at System.Linq.Enumerable.ToDictionary[TSource,TKey,TElement](IEnumerable`1 source, Func`2 keySelector, Func`2 elementSelector, IEqualityComparer`1 comparer)
   at System.Linq.Enumerable.ToDictionary[TSource,TKey,TElement](IEnumerable`1 source, Func`2 keySelector, Func`2 elementSelector)
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

We then escalate to the `.DistinctBy` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.enumerable.distinctby?view=net-8.0)]:

In [11]:
new []
{
    new KeyValuePair<string, string>("one", "uno"),
    new KeyValuePair<string, string>("two", "dos"),
    new KeyValuePair<string, string>("one", "1"),
}
.DistinctBy(kv => kv.Key)
.ToDictionary(kv => kv.Key, kv => kv.Value)

key,value
one,uno
two,dos


Now we can see how we can lose data ‘silently’ (not throwing an exception) based on document order:

In [12]:
new []
{
    new KeyValuePair<string, string>("one", "1"),
    new KeyValuePair<string, string>("one", "uno"),
    new KeyValuePair<string, string>("two", "dos"),
}
.DistinctBy(kv => kv.Key)
.ToDictionary(kv => kv.Key, kv => kv.Value)

key,value
one,1
two,dos


## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼